In [6]:
from platform import python_version
import tensorflow as tf

print(tf.test.is_gpu_available())
print(python_version())

True
3.7.5


In [7]:
import os
import numpy as np
from os import listdir
from PIL import Image
import time
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from keras import backend as K
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [39]:
def tensor_to_array(tensor1):
    return tensor1.numpy()

In [60]:
path="datasets/child_PreGen_data/"
randomiser = np.random.RandomState(123)

In [63]:
for r, d, f in os.walk(path):
    all_child = f

In [70]:
def generate_batch(child_batch):
    np_images=[]
    for child in child_batch:
        res = np.array(Image.open(path+child))
        np_images.append(res)
    return np_images

In [71]:
all_child

['50_cent_1382085219_084_1.png',
 '50_cent_1382085244_062_1.png',
 '50_cent_Luglio_02811_1.png',
 '50_cent_Rappers_00418_1.png',
 'aaliyah_Gennaio_02605_1.png',
 'aaron_paul_Agosto_01539_2.png',
 'aaron_paul_Breaking_Bad_00373_2.png',
 'abigail_breslin_Abigail-Breslin1_2.png',
 'abigail_breslin_images_2.png',
 'adam_levine_AdamLevine8thGradeCredit_1.png',
 'adam_levine_adam_levine_1.png',
 'adam_sandler_93510052_2.png',
 'adam_sandler_Settembre_03418_2.png',
 'adele_1351579506_celebrities-childhood-present-2_2.png',
 'adele_celebs-then-now-young-old-4_2.png',
 'adele_Maggio_04623_1.png',
 'adolf_hitler_Aprile_02786_2.png',
 'adolf_hitler_Politicians_00793_2.png',
 'adriana_lima_5b8bb6fdfa8e0cbd3422f0efb3223166_1.png',
 'adriana_lima_Adriana-Lima-Victorias-Angel-Child_3.png',
 'adriana_lima_adriana_lima_tumblr_luxu3zr_Ey31r6jy6zo1_500_1_add.png',
 'adriana_lima_original_1.png',
 'adriana_lima_young_adriana_lima_cm_20120321_00514_002_5.png',
 'adriano_pantaleo_adriano-pantaleo_1.png',
 '

In [72]:
img_size = 64
mean = 0.
std_dev = 0.02
lr = 0.00001
b1 = 0.9
b2 = 0.99
sd_random_normal_init = 0.02

EPOCHS = 300
batch = 64

In [73]:
def pre_gen_upsample(filters, size,apply_batchnorm = False):
    initializer = tf.random_normal_initializer(mean, std_dev)

    result = tf.keras.Sequential()
    result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                   use_bias=False))
    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())
        result.add(tf.keras.layers.ELU())

    return result

In [74]:
OUTPUT_CHANNELS = 3

In [75]:
def Pre_Generator():
    
    up_stack_noise = [
    pre_gen_upsample(64,4,apply_batchnorm=True), #8x8x64
    pre_gen_upsample(64,4,apply_batchnorm=True), #16x16x64
    pre_gen_upsample(32,4,apply_batchnorm=True)  #32x32x32
    ]
    
    initializer = tf.random_normal_initializer(mean, sd_random_normal_init)
    last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')
    
    noise = tf.keras.layers.Input(shape=(4,4,64))
    
    x = noise
    for up in up_stack_noise:
        x = up(x)
#     print(x.shape)
    output = last(x)
#     print(output.shape)
    return tf.keras.Model(inputs=noise, outputs=output)

In [76]:
pre_gen = Pre_Generator()

In [77]:
def pre_disc_downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(mean, std_dev) 
  
    result = tf.keras.Sequential()
    result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer,
                             use_bias=False))

    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())
        
    result.add(tf.keras.layers.LeakyReLU(alpha = 0.2))

    return result

In [78]:
def Pre_Discriminator():
    
    down_stack_image = [
    pre_disc_downsample(32,4,apply_batchnorm=False), #32x32x32
    pre_disc_downsample(64,4,apply_batchnorm=True), #16x16x64
    pre_disc_downsample(64,4,apply_batchnorm=True),  #8x8x64
    pre_disc_downsample(256,4,apply_batchnorm=False) #4x4x256
        
    ]
    
    initializer = tf.random_normal_initializer(mean, sd_random_normal_init)
    last = tf.keras.layers.Conv2DTranspose(1, 4,
                                         strides=1,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='sigmoid')
    
    image = tf.keras.layers.Input(shape=(64,64,3))
    print(image.shape)
    x = image
    for down in down_stack_image:
        x = down(x)
        print(x.shape)
    output = last(x)
#     print(output.shape)

    return tf.keras.Model(inputs=image, outputs=output)

In [79]:
pre_disc = Pre_Discriminator()

(None, 64, 64, 3)
(None, 32, 32, 32)
(None, 16, 16, 64)
(None, 8, 8, 64)
(None, 4, 4, 256)


In [80]:
bce = tf.keras.losses.BinaryCrossentropy()

In [81]:
pre_gen_optimizer = tf.keras.optimizers.Adam(learning_rate = lr, beta_1=b1, beta_2= b2)
pre_disc_optimizer = tf.keras.optimizers.Adam(learning_rate = lr, beta_1=b1, beta_2= b2)

In [89]:
def train_PreGAN(noise_batch, target_batch, b_size):
    with tf.GradientTape() as pre_gen_tape ,tf.GradientTape() as pre_disc_tape:
        
        pre_gen_outputs = pre_gen(noise_batch, training=True)
        disc_pre_gen_out = pre_disc(pre_gen_outputs, training=True)
        disc_target_out = pre_disc(target_batch,training=True)
        ones = tf.ones_like(disc_pre_gen_out)
        zeros = tf.zeros_like(disc_pre_gen_out)
        
        disc_loss = bce(ones,disc_target_out) + bce(zeros,disc_pre_gen_out)
        gen_loss = bce(ones,disc_pre_gen_out)
        
    
    print("GEN_LOSS: ",tensor_to_array(gen_loss))
    print("DISC_LOSS: ",tensor_to_array(disc_loss))
    
    #calculate gradients
    pre_gen_gradients = pre_gen_tape.gradient(gen_loss,pre_gen.trainable_variables)
    pre_disc_gradients = pre_disc_tape.gradient(disc_loss,pre_disc.trainable_variables)
    
    #apply gradients on optimizer
    pre_gen_optimizer.apply_gradients(zip(pre_gen_gradients,pre_gen.trainable_variables))
    pre_disc_optimizer.apply_gradients(zip(pre_disc_gradients,pre_disc.trainable_variables))

In [90]:
def fit_encoder(all_child, epochs, batch):
    for epoch in range(epochs):
        print("______________________________EPOCH %d_______________________________"%(epoch+1))
        start = time.time()
        for i in range(len(all_child)//batch):
            batch_data = np.asarray(generate_batch(all_child[i*batch:(i+1)*batch]))
            child_batch= batch_data / 255 * 2 -1
            
            
            print("Generated batch", batch_data.shape)
            
            noise = tf.random.normal((batch,4,4,64),mean= 0.0, stddev=1.0)

            train_PreGAN(noise, child_batch,batch)
            
            print("Trained for batch %d/%d"%(i+1,(len(train_ds)//batch)))
    print("______________________________TRAINING COMPLETED_______________________________")

In [ ]:
pre_gen = Pre_Generator()
pre_disc = Pre_Discriminator()
with tf.device('/gpu:0'):
    fit_encoder(all_child, EPOCHS,batch)

(None, 64, 64, 3)
(None, 32, 32, 32)
(None, 16, 16, 64)
(None, 8, 8, 64)
(None, 4, 4, 256)
______________________________EPOCH 1_______________________________
Generated batch (64, 64, 64, 3)
